# Setup

In [ ]:
conda create --name recipe1m pytorch=3
sorce activate recipe1m


# Train用データの用意

## 生データの用意
`layer1.json` （`ingredients`, `partition`, `title`, `id`, `instructions`）、 <br>
`layer2.json` （`id`、 `images`）、<br>
`det_ingrs.json` （`valid`、 `id`、 `ingredients`） を用意する。

In [1]:
import os
SUFFIX = "test"
DATA_PATH = "../data/" + SUFFIX + '/'
os.environ['DATA_PATH'] = DATA_PATH
os.environ['SUFFIX'] = SUFFIX

## bigram の生成
料理名 title について、頻度順のbi-gram (`bigram-suffix.pkl`） と 料理名一覧（`titles-suffix.txt`） を生成

`scipy` のバージョンが 1.1.0 である必要がある。JupyterLab　上ではうまく動かなかったので、これはターミナル上で叩くことにした <br>
→ 動かしたいカーネル上で jupyter-lab　を起動したら無事うまくいくようになった

In [7]:
!python bigrams.py --crtbgrs -dataset=$DATA_PATH -suffix=$SUFFIX
python bigrams.py --crtbgrs -dataset=../data/test/ -suffix=test

1.1.0
Loading dataset.
Creating bigrams...


## word2vecのtrain
料理手順 instructions について、token化した上でword2vecをtrainする。 <br>

### tokenize
`layer1.json`　の instructions を token化。 `tokenized_instructions_train.txt` を生成。

In [11]:
!python tokenize_instructions.py -dataset=$DATA_PATH -partition=train
!python tokenize_instructions.py -dataset=$DATA_PATH -partition=val
!python tokenize_instructions.py -dataset=$DATA_PATH -partition=test

python tokenize_instructions.py -dataset=../data/test/ -partition=train
python tokenize_instructions.py -dataset=../data/test/ -partition=val
python tokenize_instructions.py -dataset=../data/test/ -partition=test

Saving tokenized here: ../data/test/tokenized_instructions_train.txt
1400it [00:00, 97870.36it/s]
0.03250312805175781 seconds.
Number of unique ingredients 1936
Saving tokenized here: ../data/test/tokenized_instructions_val.txt
1400it [00:00, 385429.97it/s]
0.013836383819580078 seconds.
Number of unique ingredients 751
Saving tokenized here: ../data/test/tokenized_instructions_test.txt
1400it [00:00, 391781.80it/s]
0.013638734817504883 seconds.
Number of unique ingredients 747


### word2vec
`tokenized_instructions_train.txt` を用いてword2vecのtrain。 `vocab.bin` を生成。

In [36]:
# !cd ~/workspace/joint-embedding/im2recipe-Pytorch/scripts
# !wget https://storage.googleapis.com/google-code-archive-source/v2/code.google.com/word2vec/source-archive.zip
# !unzip source-archive.zip
# !cd word2vec/trunk/
# !make word2vec # なんかこれもターミナルでやったらできた
!mv word2vec/ _word2vec/

In [37]:
!cd ~/workspace/joint-embedding/im2recipe-Pytorch/scripts
os.environ['SOURCE'] = DATA_PATH + "tokenized_instructions_train.txt"
os.environ['TARGET'] = DATA_PATH + "vocab.bin"
!./_word2vec/trunk/word2vec -hs 1 -negative 0 -window 10 -cbow 0 -iter 10 -size 300 -binary 1 -min-count 20 -threads 20 -train $SOURCE -output $TARGET

Starting training using file ../data/test/tokenized_instructions_train.txt
Vocab size: 733
Words in train file: 112025


### vocab の確認
`vocab.bin` を可視化する。

In [4]:
os.environ['VOCAB'] = DATA_PATH + "vocab.txt"
os.environ['TARGET'] = DATA_PATH + "vocab.bin"
!python get_vocab.py $TARGET

Writing to ../data/test/vocab.txt...
done


## classes.pkl の生成
`food101_classes_renamed.txt` が見つからなかったので、 `food_classes.txt` を利用

In [9]:
!python bigrams.py --nocrtbgrs -dataset=$DATA_PATH -suffix=$SUFFIX -vocab=$VOCAB

1.1.0
Loading dataset.
Loading ingr vocab.
1


## instructions の embedding
`layer1.json` の `instructions` について skip-thoughts でベクトル化したもの = `skip-instructions` を生成するのだが、Pytorchのコードが公開されていないため、BERTによるembeddingに置き換える。<br>
GPUサーバにログインして以下を実行。必要があればDATASETのPATHをファイルを直接書き換えて変更

In [ ]:
### Run on GPU
!bert-embedding.py 

## make LMDB
mk_dataset.py は parameter 参照時に ../args.py を見ているので注意

In [6]:
### Run on GPU
!python mk_dataset.py --bert --dataset=$DATA_PATH --suffix=SUFFIX

1.1.0
Loading skip-thought vectors...
Traceback (most recent call last):
  File "mk_dataset.py", line 68, in <module>
    st_vecs_train = get_st('train')
  File "mk_dataset.py", line 37, in get_st
    info = pickle.load(f)
  File "/home/acb11910at/anaconda3/envs/scipy/lib/python3.7/site-packages/torch/storage.py", line 134, in _load_from_bytes
    return torch.load(io.BytesIO(b))
  File "/home/acb11910at/anaconda3/envs/scipy/lib/python3.7/site-packages/torch/serialization.py", line 529, in load
    return _legacy_load(opened_file, map_location, pickle_module, **pickle_load_args)
  File "/home/acb11910at/anaconda3/envs/scipy/lib/python3.7/site-packages/torch/serialization.py", line 702, in _legacy_load
    result = unpickler.load()
  File "/home/acb11910at/anaconda3/envs/scipy/lib/python3.7/site-packages/torch/serialization.py", line 665, in persistent_load
    deserialized_objects[root_key] = restore_location(obj, location)
  File "/home/acb11910at/anaconda3/envs/scipy/lib/python3.7/si

## Training
python train.py <br>
--img_path /path/to/images/ <br>
--data_path /path/to/lmdbs/ <br>
--ingrW2V /path/to/w2v/vocab.bin <br>
--snapshots snapshots/ <br>
--valfreq 10 <br>

In [ ]:
qrsh -g gcb50373 -l rt_G.small=1 -l h_rt=01:00:00
conda activate scipy
cd workspace/joint-embedding/im2recipe-Pytorch/

#### 1. 作ったLMDBで実験

In [ ]:
### run on GPU
python train.py \
--img_path /groups1/gcb50373/dataset/recipe1M/origin/images/ \
--data_path data/test/ \
--ingrW2V data/test/vocab.bin \
--snapshots snapshots/ \
--stDim 768

#### エラーメッセージの内容
/tmp/pip-req-build-ufslq_a9/aten/src/THC/THCTensorIndex.cu:361: void indexSelectLargeIndex(TensorInfo<T, IndexType>, TensorInfo<T, IndexType>, TensorInfo<long, IndexType>, int, int, IndexType, IndexType, long) [with T = float, IndexType = unsigned int, DstDim = 2, SrcDim = 2, IdxDim = -2, IndexIsMajor = true]: block: [210,0,0], thread: [96,0,0] Assertion `srcIndex < srcSelectDimSize` failed.

####  2. ダウンロードしたLMDBで実験
前やった時はできなかったはずなのに動いた

In [ ]:
python train.py \
--img_path /groups1/gcb50373/dataset/recipe1M/origin/images/ \
--data_path /groups1/gcb50373/dataset/recipe1M/origin/lmdb/ \
--ingrW2V /groups1/gcb50373/dataset/recipe1M/origin/vocab.bin \
--snapshots snapshots/ \